### Importing libraries

In [31]:
import pandas as pd
import numpy as np
import time 

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

### Getting target list 

In [32]:
all_albums = pd.read_csv('../data/clean/all_550_albums_clean.csv', encoding='latin1')

all_albums.head(3)

,toprankingorder,album,artist,year,album_artist,horriblerankingorder,rationale
0,1.0,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,1967,Sgt. Pepper's Lonely Hearts Club Band - The Be...,NaN,NaN
1,2.0,Pet Sounds,The Beach Boys,1966,Pet Sounds - The Beach Boys,NaN,NaN
2,3.0,Revolver,The Beatles,1966,Revolver - The Beatles,NaN,NaN


#### list of albums

In [33]:
albums = all_albums['album'].unique().tolist()

In [34]:
#some had the same name (e.g. Greatest Hits, Let It Be)

len(albums)

547

#### list of albums_artists 

In [35]:
album_artist = all_albums['album_artist'].unique().tolist()

In [36]:
len(album_artist)

550

### Spotify Connection 

In [37]:
#%pip install pyarrow
#%pip install joblib
#%pip install tqdm
#%pip install spotipy
#%pip install python-dotenv
#%pip install requests 

In [38]:
import requests as rq
import pyarrow
from joblib import Parallel, delayed
from tqdm import tqdm
import pymongo
from passwords import *

In [39]:
# Obtener el TOKEN

AUTH_URL = "https://accounts.spotify.com/api/token"

creds = {
    "grant_type": "client_credentials",
    "client_id": Client_ID,
    "client_secret": Client_secret
}

response = rq.post(AUTH_URL, 
                         data=creds, 
                         headers={"Content-Type": "application/x-www-form-urlencoded"})


TOKEN = response.json()['access_token']

# Conexión con la API

url = "https://api.spotify.com/"

headers = {"Authorization": f'Bearer {TOKEN}'}

response = rq.get(url,headers=headers)
response

<Response [200]>

### Getting Album Information 

In [40]:
albums[0:3]

["Sgt. Pepper's Lonely Hearts Club Band", 'Pet Sounds', 'Revolver']

In [41]:
#understanding the json 

In [42]:
url = 'https://api.spotify.com/v1/search?q=album%3Apet+sounds&type=album&limit=50&offset=0'

In [43]:
res = rq.get(url, headers=headers)
res.status_code 

200

In [44]:
data = res.json()

In [45]:
type(data)

dict

In [46]:
data.keys()

dict_keys(['albums'])

In [47]:
data['albums'].keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [48]:
data['albums']['items'][0].keys()

dict_keys(['album_type', 'artists', 'available_markets', 'external_urls', 'href', 'id', 'images', 'name', 'release_date', 'release_date_precision', 'total_tracks', 'type', 'uri'])

In [49]:
#quiero el id del album en Spotify 
data['albums']['items'][0]['id']

'6GphKx2QAPRoVGWE9D7ou8'

In [50]:
#quiero el nombre del album en Spotify
data['albums']['items'][0]['name']

'Pet Sounds (Original Mono & Stereo Mix)'

In [51]:
#quiero el nombre del artista del album 
data['albums']['items'][0]['artists'][0]['name']

'The Beach Boys'

In [52]:
#quiero el id del artista del album 
data['albums']['items'][0]['artists'][0]['id']

'3oDbviiivRWhXwIE8hxkVV'

In [53]:
#quiero el release_date del album 
data['albums']['items'][0]['release_date']

'1966-06-16'

In [54]:
#quiero el total de tracks del album 
data['albums']['items'][0]['total_tracks']

27

In [55]:
# Replace spaces with the '+' character in the album names
insertquery = [x.replace(' ', '+') for x in albums]

# Construct a list of URLs for querying album information
album_urls = [f'https://api.spotify.com/v1/search?q=album%3A{album}&type=album&limit=50&offset=0' for album in insertquery]

# Create a list of dictionaries with album information and corresponding URLs
album_url_list = [{'album': album, 'url': url} for album, url in zip(albums, album_urls)]

In [56]:
album_url_list[1]

{'album': 'Pet Sounds',
 'url': 'https://api.spotify.com/v1/search?q=album%3APet+Sounds&type=album&limit=50&offset=0'}

In [57]:
# List to store the retrieved album data
album_data_list = []

# Function to search and retrieve album information
def search_spotify_info(album_url):
    time.sleep(0.30)  # Pause between queries to avoid getting banned

    try:
        response = rq.get(album_url['url'], headers=headers)
        response.raise_for_status()  # Check for request errors
        data = response.json()
        rjson = data.get('albums', {}).get('items', [])

        for index, item in enumerate(rjson):
            if item['name'].lower() == album_url['album'].lower():
                album_data_list.append({
                    'albumnamers': album_url['album'],
                    'albumidspotify': item['id'],  # Spotify ID for the album
                    'albumnamespotify': item['name'],
                    'artistnamespotify': item['artists'][0]['name'],
                    'artistidspotify': item['artists'][0]['id'],
                    'releasedate': item['release_date'],
                    'totaltracks': item['total_tracks']
                })
                
                return rjson[index]

        # If the album is not found, raise an exception to return None
        raise Exception("Album not found")
    
    except Exception as e:
        print(f"An error occurred: {e}")
        album_data_list.append({
                    'albumnamers': None, 
                    'albumidspotify': None, 
                    'albumnamespotify': None, 
                    'artistnamespotify': None,
                    'artistidspotify': None,
                    'releasedate': None,
                    'totaltracks': None 
        })
        return {
            'album_type': None,
            'artists': None,
            'available_markets': None,
            'external_urls': None,
            'href': None,
            'id': None,
            'images': None,
            'name': None,
            'release_date': None,
            'release_date_precision': None,
            'total_tracks': None,
            'type': None,
            'uri': None
        }
    
# Execute the function for each album URL
for album_url in album_url_list:
    search_spotify_info(album_url)

# Create a DataFrame from the list of dictionaries
dataalbums = pd.DataFrame(album_data_list)

An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: Album not found
An error occurred: A

In [72]:
dataalbums.dropna(inplace=True)

In [73]:
dataalbums.shape

(443, 7)

In [74]:
dataalbums.head()

,albumnamers,albumidspotify,albumnamespotify,artistnamespotify,artistidspotify,releasedate,totaltracks
0,Sgt. Pepper's Lonely Hearts Club Band,1x1jpjDbetGqX0IKCUIBNj,Sgt. Pepper's Lonely Hearts Club Band,Bloco do Sargento Pimenta,3wGWCP3E3tYqj5memYV9Vq,2017-12-08,13.0
1,Pet Sounds,2CNEkSE8TADXRT2AzcEt1b,Pet Sounds,The Beach Boys,3oDbviiivRWhXwIE8hxkVV,1966-05-16,13.0
2,Revolver,0T1sskJDoybYGvPU5aw5Cf,REVOLVER,Lil Darkie,62F9BiUmjqeXbBztCwiX1U,2020-06-15,1.0
3,Highway 61 Revisited,6YabPKtZAjxwyWbuO9p4ZD,Highway 61 Revisited,Bob Dylan,74ASZWbe4lXaubB36ztrGX,1965-08-30,9.0
4,Rubber Soul,5TIQEIzrI6RQfUVQ5Y571D,Rubber Soul,The Beatles Complete On Ukulele,5o723EMxNulM5ydXRh7Qkk,2020-07-28,16.0


In [75]:
dataalbums.to_csv('../data/clean/api_albums_clean.csv', index=False)